In [ ]:
'''
:@Author: KIKO_KEKE
:@Date: 2024/4/9 12:59:27
:@LastEditors: KIKO_KEKE
:@LastEditTime: 2024/4/9 12:59:27
:Description: 
:Copyright: Copyright (©)}) 2024 KIKO_KEKE. All rights reserved.
'''


In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import os
from selenium.webdriver.common.action_chains import ActionChains
import urllib.request
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
venue_name = "Davis Wade Stadium"


In [2]:
chrome_options = Options()
# 设置浏览器首选语言为英语
chrome_options.add_argument("--lang=en")
# 使用配置的选项初始化WebDriver
driver = webdriver.Chrome(options=chrome_options)
# 打开谷歌地图搜索页面
driver.get("https://www.google.com/maps")
time.sleep(1)  # 等待页面加载

# 在搜索框中输入体育场馆名称并搜索
search_box = driver.find_element(By.CSS_SELECTOR, "input#searchboxinput")
search_box.send_keys(venue_name)
search_button = driver.find_element(By.CSS_SELECTOR, "button#searchbox-searchbutton")
search_button.click()
time.sleep(5)  # 等待搜索结果加载
# 点击评价
wait = WebDriverWait(driver, 300)
review_section = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[jsaction='pane.reviewChart.moreReviews']")))
review_section.click()
time.sleep(2)  # 等待页面加载
# 初始化ActionChains
actions = ActionChains(driver)
original_window = driver.current_window_handle  # 保存原始窗口的句柄
all_reviews = []
# 获取评价评分，使用简化的CSS选择器
average_rating_element = driver.find_element(By.CSS_SELECTOR, ".jANrlb > .fontDisplayLarge")
average_rating = average_rating_element.text if average_rating_element else "无特定评分"
# 滚动加载评论
def load_more_reviews():
    max_retries = 10  # 设置最大重试次数
    retry_count = 0  # 初始化重试计数器

    while True:
        reviews = driver.find_elements(By.CSS_SELECTOR, "div.jftiEf")
        current_review_count = len(reviews)
        print(f"Current number of reviews: {current_review_count}")

        # 执行页面操作尝试加载更多评论...
        reviews = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.jftiEf")))
        if not reviews:
            print("No reviews found.")
            break

        # 将最后一条评论滚动到视图中
        last_review = reviews[-1]
        driver.execute_script("arguments[0].scrollIntoView();", last_review)

        # 点击最后一条评论的星级处获得焦点
        dot = driver.find_elements(By.CSS_SELECTOR, ".DU9Pgb .NhBTye")
        actions.move_to_element(dot[-1]).click().perform()
        
        # 检查是否打开了新窗口，并切换回原始窗口
        if len(driver.window_handles) > 1:
            for window_handle in driver.window_handles:
                if window_handle != original_window:
                    driver.switch_to.window(window_handle)
                    driver.close()
            driver.switch_to.window(original_window)

        # 尝试加载更多评论
        actions.send_keys(Keys.PAGE_DOWN).perform()
        try:
            # 等待评论数量增加
            wait.until(lambda d: len(d.find_elements(By.CSS_SELECTOR, "div.jftiEf")) > current_review_count)
            print("New reviews loaded.")
            reviews = driver.find_elements(By.CSS_SELECTOR, "div.jftiEf")  # 更新评论列表
            retry_count = 0  # 成功加载后重置重试计数器
        except TimeoutException:
            print(f"Retry attempt: {retry_count}")
            retry_count += 1  # 增加重试计数器
            if retry_count >= max_retries:
                print("Maximum retries reached, no new reviews loaded.")
                break  # 达到最大重试次数后退出循环
    return reviews    
# 调用函数加载评论

endreviews=load_more_reviews()


Current number of reviews: 10
New reviews loaded.
Current number of reviews: 20
New reviews loaded.
Current number of reviews: 30
New reviews loaded.
Current number of reviews: 40
New reviews loaded.
Current number of reviews: 50
New reviews loaded.
Current number of reviews: 60
New reviews loaded.
Current number of reviews: 70
New reviews loaded.
Current number of reviews: 80
New reviews loaded.
Current number of reviews: 90
New reviews loaded.
Current number of reviews: 100
New reviews loaded.
Current number of reviews: 110
New reviews loaded.
Current number of reviews: 120
New reviews loaded.
Current number of reviews: 130
New reviews loaded.
Current number of reviews: 140
New reviews loaded.
Current number of reviews: 150
New reviews loaded.
Current number of reviews: 160
New reviews loaded.
Current number of reviews: 170
New reviews loaded.
Current number of reviews: 180
New reviews loaded.
Current number of reviews: 190
New reviews loaded.
Current number of reviews: 200
New revie

In [4]:
#load_more_reviews()
#endreviews = driver.find_elements(By.CSS_SELECTOR, "div.jftiEf")
for review in endreviews:
    reviewer_name = review.find_element(By.CSS_SELECTOR, ".d4r55").text  # 获取评论者名字
    # 直接获取评论者的评价信息
    author_reviews_info_element = review.find_elements(By.CSS_SELECTOR, ".RfnDt") 
    author_reviews_info=author_reviews_info_element[0].text if author_reviews_info_element else "无评论者信息"
    # 获取评论星级
    rating_elements = review.find_elements(By.CSS_SELECTOR, "span.kvMYJc[role='img']")
    rating = rating_elements[0].get_attribute("aria-label") if rating_elements else "无评级"
    # 获取评论内容
    review_text_elements = review.find_elements(By.CSS_SELECTOR, ".MyEned")
    review_text = review_text_elements[0].text if review_text_elements else "无评论内容"
    # 获取评论发布时间
    review_datetime_elements = review.find_elements(By.CSS_SELECTOR, "span.rsqaWe")
    review_datetime_utc = review_datetime_elements[0].text if review_datetime_elements else "无发布时间"
    # 获取评论获得的点赞数
    review_likes_elements = review.find_elements(By.CSS_SELECTOR, ".Upo0Ec .znYl0")
    review_likes = review_likes_elements[0].text if review_likes_elements else "0 likes"
    # 提取所有图片链接
    image_links = []
    image_elements = review.find_elements(By.CSS_SELECTOR, ".KtCyie .Tya61d")
    for image_element in image_elements:
        style = image_element.get_attribute('style')
        url_start = style.find('url("') + 5
        url_end = style.find('");', url_start)
        image_url = style[url_start:url_end]
        image_links.append(image_url)
    review_info = {
        "name":venue_name,
        "Average_Rating":average_rating,
        "Reviewer": reviewer_name,
        "author_reviews_info":author_reviews_info,
        "Rating": rating,
        "Review": review_text,
        "review_datetime_utc":review_datetime_utc,
        "review_likes":review_likes,
        "Image Links": image_links,
    }
    all_reviews.append(review_info)
df = pd.DataFrame(all_reviews)
df.to_excel("Davis Wade Stadium.xlsx", index=False)
print ("保存成功")

保存成功


加从Excel里下载文件

In [ ]:
import pandas as pd
import requests
import os

# 假设你的数据已经保存在一个名为'Donald W. Reynolds Razorback Stadium3.xlsx'的Excel文件中
data_file = 'Davis Wade Stadium.xlsx'
# 图片保存的目录
image_dir = 'downloaded_images_Davis Wade Stadium'

# 读取Excel文件
df = pd.read_excel(data_file)

# 确保图片保存目录存在
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

def download_image(image_url, save_path):
    """下载图片并保存到指定路径，如果图片已存在则跳过"""
    if os.path.exists(save_path):
        print(f"Image already exists: {save_path}")
        return
    try:
        response = requests.get(image_url, stream=True)
        if response.status_code == 200:
            with open(save_path, 'wb') as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)
            print(f"Image saved to {save_path}")
        else:
            print(f"Failed to download image from {image_url}")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {image_url}: {e}")

# 遍历DataFrame中的每一行
for index, row in df.iterrows():
    image_links = eval(row['Image Links'])  # 假设Image Links列是字符串表示的列表
    for image_url in image_links:
        # 提取URL的最后一部分作为文件名
        filename = image_url.split('/')[-1]
        # 替换Windows文件名中不允许的字符
        filename = filename.replace('?', '_').replace('&', '_').replace('=', '_').replace(':', '_').replace('*', '_').replace('"', '_').replace('<', '_').replace('>', '_').replace('|', '_')
        # 确保文件名以.jpg结尾
        if not filename.lower().endswith('.jpg'):
            filename += '.jpg'
        save_path = os.path.join(image_dir, filename)
        download_image(image_url, save_path)
